In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import sparse
import json
import os

### This notebook calculates the parameters for the Chicago Metropolitan area
- homdeNodes: the dataset that indicates the home charger session in each trip chain
- Chain_df: trip chain
- mapping: mapping from nodes to counties

In [15]:
N_e = 11080
N_n = 1844
N_t = 96

batteryCapacity = 56.5 # kWh

# Load trip chains
with open('TripChains/Home_new_Chicago2017.txt', 'r') as f:
    homeNodes = json.load(f)

# 从 JSON 文件中读取字典
with open('TripChains/Chicago2017_Raw/Chain_df_rec.json', 'r') as f:
    Chain_df_ori = json.load(f)

Chain_df = {}
for e in range(N_e):
    Chain_df[e] = pd.DataFrame(Chain_df_ori[e], columns = ['n^dep', 'n^arr', 't^dep', 't^arr', 'Delta_s'])

# Load the weights of trip chains
q = pd.read_csv('TripChains/Chicago2017_Raw/flows.csv', index_col = 0).values.T[0]

# set initial energy in EV battery
s_init = np.zeros(N_e) + 0.5

# set the maximum power of chargers
P_max_home = 7 / batteryCapacity # AC charger: set as 7kW, convert to SoC: 7/56.5 
P_max_work = 15 / batteryCapacity # DC charger: set as 15kW, convert to SoC: 15/56.5


# get trip numbers of each trip chain
N_i = np.zeros(N_e)
for e in range(N_e):
    N_i[e] = Chain_df[e].shape[0]
N_i = N_i.astype(int)

In [5]:
# load the mapping from node No. to county No.
mapping = pd.read_csv('TripChains/county_Mapping.csv', index_col = 0)
mappings = {}
for i in range(len(mapping)):
    mappings[mapping['nodeIndex'][i]] = mapping['countyIndex'][i]
countyIndices = mapping['countyIndex'].unique()

# set index for each county
countyNodes = {}
for n in range(len(countyIndices)):
    countyNodes[countyIndices[n]] = n

# define the county names
countyName = ['Cook', 'DuPage', 'Kane', 'Lake', 'McHenry', 'Will', 'Kendall']

# home charger availability of each county
ratio_homeCharger = {
    0: 0.456,
    1: 0.609,
    2: 0.711,
    3: 0.769,
    4: 0.875,
    5: 0.864,
    6: 0.918,
}

In [6]:
# map the home node to the corresponding county index
home_node_rec = np.zeros(N_e)
for e in range(N_e):
    try:
        home_node_temp = np.where(np.array(homeNodes[e]) == 1)[0][-1]
        home_node_temp = Chain_df[e]['n^arr'].values[home_node_temp+1]
    except:
        home_node_temp = Chain_df[e]['n^arr'].values[0]
    home_node_rec[e] = int(countyNodes[mappings[home_node_temp]])

In [7]:
"""
get the parameter (we create two chains based on each original trip chain, one with home charger, the other without)

1. q_rec: weights of trip chains, shape is (2*N_e, N_t)
2. demand_cum_rec: cumulative demand of trip chains, shape is (2*N_e, N_t)
3. Location_rec: location of the EV at each time interval, shape is (2*N_e, N_t)
"""

L = np.zeros((N_t, N_t))
for i in range(N_t):
    for j in range(N_t):
        if i >= j:
            L[i, j] = 1

E_init = 0.5
E_min = np.zeros(N_t)
E_min[-1] = 0.5
p_max_rec = np.zeros((2*N_e, N_t))
vio_rec = np.zeros(2*N_e)
q_rec = np.zeros(2*N_e)

Location_rec = np.zeros((2*N_e, N_t)) - 1
demand_cum_rec = np.zeros((2*N_e, N_t))

homeParkSession_duration = {}
nonHomeParkSession_duration = {}

home_node_update = {}
nonHomeParkDuration = {}
nonHomeParkStart = {}
nonHomeParkEnd = {}
for e in range(N_e):
    if Chain_df[e]['t^arr'].values[-1] < N_t - 1:
        start_charge, end_charge = np.zeros(N_i[e]), np.zeros(N_i[e])
        start_charge[:-1] = Chain_df[e]['t^arr'].values[:-1]
        end_charge[:-1] = Chain_df[e]['t^dep'].values[1:]
        start_charge[-1] = Chain_df[e]['t^arr'].values[-1]
        end_charge[-1] = N_t
        N_i_temp = N_i[e]
        homeNodes_e = homeNodes[e]
    else:
        start_charge, end_charge = np.zeros(N_i[e]-1), np.zeros(N_i[e]-1)
        start_charge = Chain_df[e]['t^arr'].values[:-1]
        end_charge = Chain_df[e]['t^dep'].values[1:]
        N_i_temp = N_i[e] - 1
        homeNodes_e = homeNodes[e][:-1]

    start_charge = start_charge.astype(int)
    end_charge = end_charge.astype(int)
    duration_charge = end_charge - start_charge
    location_charge = Chain_df[e]['n^arr'].values.astype(int)

    demand_e = np.zeros(N_t)
    for i in range(len(start_charge)):
        demand_e[start_charge[i]] += Chain_df[e]['Delta_s'].values[i]

    demand_cum = np.cumsum(demand_e)
    demand_cum_rec[e] = demand_cum
    demand_cum_rec[N_e + e] = demand_cum

    nonHomeParkDuration[e] = []
    nonHomeParkStart[e] = []
    nonHomeParkEnd[e] = []
    nonHomeParkDuration[N_e + e] = []
    nonHomeParkStart[N_e + e] = []
    nonHomeParkEnd[N_e + e] = []

    try:
        home_node_temp = np.where(np.array(homeNodes[e]) == 1)[0][-1]
        home_node_temp = Chain_df[e]['n^arr'].values[home_node_temp+1]
    except:
        home_node_temp = Chain_df[e]['n^arr'].values[0]

    home_node_update[e] = np.zeros(N_i_temp)
    home_node_update[e][1:] = homeNodes_e
    if home_node_temp == location_charge[0]:
        home_node_update[e][0] = 1

    for i in range(N_i_temp):
        Location_rec[e, start_charge[i]: end_charge[i]] = mappings[location_charge[i]]
        Location_rec[N_e+e, start_charge[i]: end_charge[i]] = mappings[location_charge[i]]
        
        if home_node_update[e][i] == 1:
            p_max_rec[e, start_charge[i]: end_charge[i]] = P_max_home
            if duration_charge[i] not in homeParkSession_duration.keys():
                homeParkSession_duration[duration_charge[i]] = q[e]
            else:
                homeParkSession_duration[duration_charge[i]] += q[e]
        else:
            nonHomeParkDuration[e].append(duration_charge[i])
            nonHomeParkStart[e].append(start_charge[i])
            nonHomeParkEnd[e].append(end_charge[i])
            nonHomeParkDuration[N_e + e].append(duration_charge[i])
            nonHomeParkStart[N_e + e].append(start_charge[i])
            nonHomeParkEnd[N_e + e].append(end_charge[i])

            if duration_charge[i] not in nonHomeParkSession_duration.keys():
                nonHomeParkSession_duration[duration_charge[i]] = q[e]
            else:
                nonHomeParkSession_duration[duration_charge[i]] += q[e]

    E_e_home = E_init - demand_cum + L @ p_max_rec[e] * 0.25
    E_e_pub = E_init - demand_cum
    vio_rec[e] = (E_e_home - E_min).min()
    vio_rec[N_e + e] = (E_e_pub - E_min).min()

    q_rec[e] = q[e] * ratio_homeCharger[home_node_rec[e]]
    q_rec[N_e + e] = q[e] * (1 - ratio_homeCharger[home_node_rec[e]])


path_temp = 'Parameters/Chicago2017_RealCapacity/pubCharger'
isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true
if not isExists:
    os.makedirs(path_temp)

q_rec_sparse = sparse.csr_matrix(q_rec)
demand_sparse = sparse.csr_matrix(demand_cum_rec)
Location_sparse = sparse.csr_matrix(Location_rec)

# sparse.save_npz(f"{path_temp}/q_rec.npz", q_rec_sparse)
# sparse.save_npz(f"{path_temp}/demand_cum.npz", demand_sparse)
# sparse.save_npz(f"{path_temp}/Location.npz", Location_sparse)



In [8]:
"""
calculate the maximum charging power matrix under different public charger availability
p_max_rec: maximum charging power matrix, shape is (2*N_e, N_t)
"""
nonHomeParkSession_total = 0
for e in range(N_e, 2*N_e):
    nonHomeParkSession_total += len(nonHomeParkDuration[e]) * q_rec[e]

ratio_pubCharger_list = np.linspace(0.1, 1, 10)

nonHomeParkSession = 0
for ratio_pubCharger in ratio_pubCharger_list:
    ratio_current = 0
    for kk in range(100000):
        prority = np.argsort(vio_rec[N_e:])
        e = prority[0] + N_e
        demand_cum_temp = demand_cum_rec[e]
        q_actual = q_rec[e]
        try:
            i_temp = np.array(nonHomeParkDuration[e]).argmax()
            nonHomeParkDuration[e][i_temp] = -1
            p_max_rec[e, nonHomeParkStart[e][i_temp]: nonHomeParkEnd[e][i_temp]] = P_max_work
            p_max_rec[e-N_e, nonHomeParkStart[e-N_e][i_temp]: nonHomeParkEnd[e-N_e][i_temp]] = P_max_work # mirror
        except:
            q_actual = 0

        nonHomeParkSession += q_actual
        E_e = E_init - demand_cum_temp + L @ p_max_rec[e] * 0.25
        E_e_2 = E_init - demand_cum_temp + L @ p_max_rec[e-N_e] * 0.25 # mirror

        if len(nonHomeParkDuration[e]) == 0:
            vio_rec[e] = 100
            vio_rec[e-N_e] = 100
        elif np.array(nonHomeParkDuration[e]).mean() == -1:
                vio_rec[e] = 100
                vio_rec[e-N_e] = 100
        else:
            vio_rec[e] = (E_e - E_min).min()
            vio_rec[e-N_e] = (E_e_2 - E_min).min()
        ratio_current = nonHomeParkSession / nonHomeParkSession_total

        # if vio_rec.min() >= 0:
        #     print ('all chains become feasible', ratio_current)

        if ratio_current >= ratio_pubCharger:
            print (ratio_current, p_max_rec.sum(), vio_rec.min())
            break



    print(f'ratio_pubCharger: {ratio_pubCharger}, ratio_current: {ratio_current}')

    path_temp = 'Parameters/Chicago2017_RealCapacity/pubCharger'
    isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true
    if not isExists:
        os.makedirs(path_temp)

    """save the energy bounds"""
    p_max_sparse = sparse.csr_matrix(p_max_rec)
    # sparse.save_npz(f"{path_temp}/p_max_ratio={ratio_pubCharger:.1f}", p_max_sparse)


0.10002141513767389 127961.94690265499 -0.1850063999999999
ratio_pubCharger: 0.1, ratio_current: 0.10002141513767389
0.20000249563194555 167264.0707964601 -0.0974975999999999
ratio_pubCharger: 0.2, ratio_current: 0.20000249563194555
0.3000248439225222 198630.26548672572 -0.046751999999999794
ratio_pubCharger: 0.30000000000000004, ratio_current: 0.3000248439225222
0.4000782422854216 224836.99115044245 -0.0042239999999997835
ratio_pubCharger: 0.4, ratio_current: 0.4000782422854216
0.5000487301691656 228452.38938053115 0.3184708814159294
ratio_pubCharger: 0.5, ratio_current: 0.5000487301691656
0.6000808880257632 234165.66371681416 0.4362982513274344
ratio_pubCharger: 0.6, ratio_current: 0.6000808880257632
0.700035484618879 239448.31858407092 0.4819876814159293
ratio_pubCharger: 0.7000000000000001, ratio_current: 0.700035484618879
0.8000125363217137 243778.93805309746 0.5
ratio_pubCharger: 0.8, ratio_current: 0.8000125363217137
0.9000434419068222 248478.5840707965 0.5
ratio_pubCharger: 0.9